In [1]:
!pwd

/home/sagemaker-user/Text-Summarizer/notebooks


In [2]:
%cd ..

/home/sagemaker-user/Text-Summarizer


# Configuration

In [3]:
from dataclasses import dataclass

In [4]:
@dataclass
class EvalConfig:
    folder_path: str
    model_path: str
    tokenizer_path: str
    data_path: str

# Configuration Manager

In [5]:
from src.config import ConfigManager
from src.utils import create_dirs
from os.path import join

In [6]:
class ConfigManager(ConfigManager):
    def get_eval_config(self):
        direct = join(self.artifacts, self.config.evaluation.folder)
        create_dirs([direct])

        model_path = self.get_trainer_config().config_path+'/model'
        tokenizer_path = self.get_trainer_config().config_path+'/tokenizer'
        data_path = self.get_data_config().save_path

        return EvalConfig(
            folder_path=direct,
            model_path=model_path,
            tokenizer_path=tokenizer_path,
            data_path=data_path
        )

# Eval Class

In [7]:
from datasets import load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import evaluate
import torch
import json
from src import logger
from tqdm import tqdm

[2024-10-09 12:08:10,189: INFO: config: PyTorch version 2.3.1.post100 available.]
[2024-10-09 12:08:10,191: INFO: config: TensorFlow version 2.17.0 available.]


2024-10-09 12:08:11.633873: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-09 12:08:11.662321: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
class Eval:
    def __init__(self, config: EvalConfig):
        self.config = config

    def generate_batch(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def eval(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'

        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        test_data = load_from_disk(self.config.data_path, )['test']

        inp_batches = list(self.generate_batch(test_data['dialogue'], 16))
        tar_batches = list(self.generate_batch(test_data['summary'], 16))

        rouge_metric = evaluate.load('rouge')

        for inp_batch, tar_batch in tqdm(zip(inp_batches, tar_batches), total=len(inp_batches)):

            inputs = tokenizer(
                inp_batch, max_length=1024, padding='max_length', truncation=True, return_tensors="pt"
            )
            summaries = model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                length_penalty=0.8, num_beams=8, max_length=128
            )

            decoded_summaries = [
                tokenizer.decode(summary, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                for summary in summaries
            ]

            rouge_metric.add_batch(predictions=decoded_summaries, references=tar_batch)

        metrics = rouge_metric.compute()

        with open(self.config.folder_path+'/eval.json', "w") as f:
            json.dump(metrics, f, indent=4)
            logger.info(f"Evaluation file saved at: {self.config.folder_path}")

# Run the step

In [17]:
try:
    config = ConfigManager().get_eval_config()
    Eval(config).eval()
except Exception as e:
    raise e

[2024-10-09 12:51:55,496: INFO: utils: The file: params.yaml loaded successfully...]
[2024-10-09 12:51:55,498: INFO: utils: The file: config.yaml loaded successfully...]


100%|██████████| 52/52 [36:39<00:00, 42.30s/it]

[2024-10-09 13:28:38,375: INFO: rouge_scorer: Using default tokenizer.]


[2024-10-09 13:28:39,170: INFO: 1553023937: Evaluation file saved at: artifacts/eval]
